In [5]:
import json
import os
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
import time

In [6]:
openai.api_key  = ''
openai.api_type = 'azure'
openai.api_base = ''
openai.api_version = ''

In [8]:
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

def tiktoken_len(text):
    tokens=encoding.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 100,
    length_function = tiktoken_len
)

In [11]:
def create_embedding(text):
    res=[]
    count=len(text)

    for i in range(len(text)):
        embedding = openai.Embedding.create(input=text[i], engine='text-embedding-ada-002')
        embedding = embedding['data'][0]['embedding']
        res.append(embedding)
        time.sleep(1)
        print(i, '/', count, 'done')

    if (len(res) == len(text)):
        return True, res
    
    return False, []

In [9]:
files = os.listdir('../data/scrapped_data/')

In [12]:
succeeded_files = set(os.listdir('../data/all_chunked_data/'))
remaining_files = []

for file in files:
    if (file not in succeeded_files):
        remaining_files.append(file)

In [13]:
len(remaining_files)

1

In [ ]:
while (remaining_files!=[]):
    for file in remaining_files:
        if (file in succeeded_files):
            print(f"{file} already processed!")
            continue

        print(file)
        with open(f'../data/scrapped_data/{file}') as f:
            d = json.load(f)

        contents = text_splitter.split_text(d['content'].replace('\n\n', '. ', 1))
        d['content'] = contents

        try:
            done, embedding = create_embedding(d['content'])

            if(done):
                d['content_vector'] = embedding
                with open(f'../data/all_chunked_data/{file}', 'w') as f:
                    json.dump(d, f)

                succeeded_files.add(file)
                print(len(succeeded_files), 'done')

            else:
                print(f"{file} failed!")

        except Exception as ex:
            print(ex)
            time.sleep(10)

    remaining_files=[]
    succeeded_files=set(os.listdir('../data/all_chunked_data/'))

    for file in files:
        if (file not in succeeded_files):
            remaining_files.append(file)